Importy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu, kruskal
import itertools
import io
from contextlib import redirect_stdout

Wczytywanie danych

In [15]:
def load_data(file_path='paste.txt'):
    df = pd.read_csv(file_path)
    return df

Przygotowanie danych 

In [16]:
def prepare_data(df):
    # Usuwamy wiersze, gdzie Track_Time jest równe 0 (nie ukończone podejścia bez czasu)
    df = df[df['Track_Time'] > 0]
    
    df = df[df['Track_ID'] <=7]
    # Przekształcamy zmienne
    df['Track_Time_Seconds'] = df['Track_Time'] / 1000  # Konwersja z milisekund na sekundy
    
    return df

Test Wilcoxona

In [26]:
# Funkcja do przeprowadzenia testu Wilcoxona (Mann-Whitney U) dla dwóch grup z zrównoważonym samplingiem
def wilcoxon_test_balanced(data, group_col, value_col, group1, group2, track_id=None, random_seed=42):
    # Filtrowanie danych dla określonego toru, jeśli podano
    if track_id is not None:
        data = data[data['Track_ID'] == track_id]
    
    # Dzielimy dane na grupy - oryginalne niezmodyfikowane zbiory
    group1_data_orig = data[data[group_col] == group1][value_col]
    group2_data_orig = data[data[group_col] == group2][value_col]
    
    # Sprawdzamy, czy mamy wystarczającą ilość danych do przeprowadzenia testu
    if len(group1_data_orig) < 2 or len(group2_data_orig) < 2:
        return {
            'group1': group1,
            'group2': group2,
            'n1_original': len(group1_data_orig),
            'n2_original': len(group2_data_orig),
            'n_sampled': 0,
            'mean1_original': group1_data_orig.mean() if len(group1_data_orig) > 0 else np.nan,
            'mean2_original': group2_data_orig.mean() if len(group2_data_orig) > 0 else np.nan,
            'median1_original': group1_data_orig.median() if len(group1_data_orig) > 0 else np.nan,
            'median2_original': group2_data_orig.median() if len(group2_data_orig) > 0 else np.nan,
            'mean1_sampled': np.nan,
            'mean2_sampled': np.nan,
            'median1_sampled': np.nan,
            'median2_sampled': np.nan,
            'statistic': np.nan,
            'p_value': np.nan,
            'significant': np.nan
        }
    
    # Przeprowadzamy zrównoważone próbkowanie
    np.random.seed(random_seed)
    
    # Obliczamy liczność mniejszej grupy
    min_size = min(len(group1_data_orig), len(group2_data_orig))
    
    # Kopie oryginalnych danych do samplowania
    group1_data = group1_data_orig.copy()
    group2_data = group2_data_orig.copy()
    
    # Próbkujemy z większej grupy, aby zrównać liczności
    if len(group1_data) > min_size:
        group1_data = group1_data.sample(min_size, random_state=random_seed)
    elif len(group2_data) > min_size:
        group2_data = group2_data.sample(min_size, random_state=random_seed)
    
    # Przeprowadzamy test Wilcoxona (Mann-Whitney U) na zsamplowanych danych
    statistic, p_value = mannwhitneyu(group1_data, group2_data, alternative='two-sided')
    
    # Przygotowanie wyników z danymi zarówno z oryginalnych, jak i zsamplowanych zbiorów
    result = {
        'group1': group1,
        'group2': group2,
        'n1_original': len(group1_data_orig),
        'n2_original': len(group2_data_orig),
        'n_sampled': min_size,
        'mean1_original': group1_data_orig.mean(),
        'mean2_original': group2_data_orig.mean(),
        'median1_original': group1_data_orig.median(),
        'median2_original': group2_data_orig.median(),
        'mean1_sampled': group1_data.mean(),
        'mean2_sampled': group2_data.mean(),
        'median1_sampled': group1_data.median(),
        'median2_sampled': group2_data.median(),
        'statistic': statistic,
        'p_value': p_value,
        'significant': p_value < 0.05
    }
    
    return result

In [27]:
# Funkcja do przeprowadzenia testów Wilcoxona dla wszystkich par grup z zrównoważonym samplingiem
def pairwise_wilcoxon_tests_balanced(data, group_col, value_col, track_id=None, random_seed=42):
    # Filtrowanie danych dla określonego toru, jeśli podano
    if track_id is not None:
        data = data[data['Track_ID'] == track_id]
    
    # Pobieramy unikalne grupy
    groups = sorted(data[group_col].unique())
    
    # Tworzymy wszystkie możliwe pary grup
    group_pairs = list(itertools.combinations(groups, 2))
    
    # Przeprowadzamy test dla każdej pary
    results = []
    for group1, group2 in group_pairs:
        result = wilcoxon_test_balanced(data, group_col, value_col, group1, group2, track_id, random_seed)
        results.append(result)
    
    return results

# Funkcja do przeprowadzenia testu Kruskala-Wallisa dla więcej niż dwóch grup
def kruskal_test(data, group_col, value_col, track_id=None):
    # Filtrowanie danych dla określonego toru, jeśli podano
    if track_id is not None:
        data = data[data['Track_ID'] == track_id]
    
    # Pobieramy unikalne grupy
    groups = sorted(data[group_col].unique())
    
    # Sprawdzamy, czy mamy wystarczającą ilość grup
    if len(groups) < 2:
        return {
            'groups': groups,
            'statistic': np.nan,
            'p_value': np.nan,
            'significant': np.nan
        }
    
    # Przygotowujemy dane dla każdej grupy
    group_data = [data[data[group_col] == group][value_col].values for group in groups]
    
    # Sprawdzamy, czy mamy wystarczającą ilość danych w każdej grupie
    if any(len(data) < 1 for data in group_data):
        return {
            'groups': groups,
            'statistic': np.nan,
            'p_value': np.nan,
            'significant': np.nan
        }
    
    # Przeprowadzamy test Kruskala-Wallisa
    statistic, p_value = kruskal(*group_data)
    
    # Przygotowanie wyników
    result = {
        'groups': groups,
        'statistic': statistic,
        'p_value': p_value,
        'significant': p_value < 0.05
    }
    
    return result

# Funkcja do analizy wpływu wieku i relacji na wyniki
def analyze_performance(df, random_seed=42):
    # Analizujemy tylko ukończone podejścia
    completed_df = df[df['Track_Completed'] == True]
    
    # Unikalne identyfikatory torów
    track_ids = sorted(df['Track_ID'].unique())
    
    results = {
        'age_groups': {},
        'relationships': {}
    }
    
    # Analiza dla wszystkich danych (bez podziału na tory)
    print("Analiza dla wszystkich torów łącznie:")
    print("\nTest Kruskala-Wallisa dla grup wiekowych (Age_Group_X):")
    kruskal_result_age_x = kruskal_test(completed_df, 'Age_Group_X', 'Track_Time_Seconds')
    print(f"Statystyka: {kruskal_result_age_x['statistic']:.2f}, p-wartość: {kruskal_result_age_x['p_value']:.4f}, Istotne: {kruskal_result_age_x['significant']}")
    
    print("\nTest Kruskala-Wallisa dla grup wiekowych (Age_Group_Y):")
    kruskal_result_age_y = kruskal_test(completed_df, 'Age_Group_Y', 'Track_Time_Seconds')
    print(f"Statystyka: {kruskal_result_age_y['statistic']:.2f}, p-wartość: {kruskal_result_age_y['p_value']:.4f}, Istotne: {kruskal_result_age_y['significant']}")
    
    print("\nTest Kruskala-Wallisa dla relacji (Relationship_Name):")
    kruskal_result_rel = kruskal_test(completed_df, 'Relationship_Name', 'Track_Time_Seconds')
    print(f"Statystyka: {kruskal_result_rel['statistic']:.2f}, p-wartość: {kruskal_result_rel['p_value']:.4f}, Istotne: {kruskal_result_rel['significant']}")
    
    # Analizujemy każdy tor osobno
    for track_id in track_ids:
        print(f"\n\n=== Analiza dla toru {track_id} ===")
        track_df = completed_df[completed_df['Track_ID'] == track_id]
        
        # Sprawdzamy, czy mamy wystarczającą ilość danych dla tego toru
        if len(track_df) < 5:
            print(f"Za mało danych dla toru {track_id} (n={len(track_df)}). Pomijam.")
            continue
        
        # Test Kruskala-Wallisa dla grup wiekowych
        print("\nTest Kruskala-Wallisa dla grup wiekowych (Age_Group_X):")
        kruskal_result_age_x = kruskal_test(track_df, 'Age_Group_X', 'Track_Time_Seconds')
        print(f"Statystyka: {kruskal_result_age_x['statistic']:.2f}, p-wartość: {kruskal_result_age_x['p_value']:.4f}, Istotne: {kruskal_result_age_x['significant']}")
        
        print("\nTest Kruskala-Wallisa dla grup wiekowych (Age_Group_Y):")
        kruskal_result_age_y = kruskal_test(track_df, 'Age_Group_Y', 'Track_Time_Seconds')
        print(f"Statystyka: {kruskal_result_age_y['statistic']:.2f}, p-wartość: {kruskal_result_age_y['p_value']:.4f}, Istotne: {kruskal_result_age_y['significant']}")
        
        print("\nTest Kruskala-Wallisa dla relacji (Relationship_Name):")
        kruskal_result_rel = kruskal_test(track_df, 'Relationship_Name', 'Track_Time_Seconds')
        print(f"Statystyka: {kruskal_result_rel['statistic']:.2f}, p-wartość: {kruskal_result_rel['p_value']:.4f}, Istotne: {kruskal_result_rel['significant']}")
        
        # Szczegółowe testy Wilcoxona dla par grup z zrównoważonym samplingiem
        print("\nSzczegółowe testy Wilcoxona z zrównoważonym samplingiem dla par grup wiekowych (Age_Group_X):")
        wilcoxon_results_age_x = pairwise_wilcoxon_tests_balanced(track_df, 'Age_Group_X', 'Track_Time_Seconds', track_id, random_seed)
        for res in wilcoxon_results_age_x:
            if np.isnan(res['p_value']):
                print(f"{res['group1']} vs {res['group2']}: Za mało danych (n1={res['n1_original']}, n2={res['n2_original']})")
            else:
                print(f"{res['group1']} vs {res['group2']}: Statystyka={res['statistic']:.2f}, p-wartość={res['p_value']:.4f}, Istotne={res['significant']}")
                print(f"  Próbki: Oryginalne (n1={res['n1_original']}, n2={res['n2_original']}), Po zrównoważeniu: n={res['n_sampled']}")
                print(f"  Średnie czasy (oryginalne): {res['group1']}={res['mean1_original']:.2f}s, {res['group2']}={res['mean2_original']:.2f}s")
                print(f"  Mediany czasów (oryginalne): {res['group1']}={res['median1_original']:.2f}s, {res['group2']}={res['median2_original']:.2f}s")
                print(f"  Średnie czasy (zsamplowane): {res['group1']}={res['mean1_sampled']:.2f}s, {res['group2']}={res['mean2_sampled']:.2f}s")
                print(f"  Mediany czasów (zsamplowane): {res['group1']}={res['median1_sampled']:.2f}s, {res['group2']}={res['median2_sampled']:.2f}s")
        
        print("\nSzczegółowe testy Wilcoxona z zrównoważonym samplingiem dla par grup wiekowych (Age_Group_Y):")
        wilcoxon_results_age_y = pairwise_wilcoxon_tests_balanced(track_df, 'Age_Group_Y', 'Track_Time_Seconds', track_id, random_seed)
        for res in wilcoxon_results_age_y:
            if np.isnan(res['p_value']):
                print(f"{res['group1']} vs {res['group2']}: Za mało danych (n1={res['n1_original']}, n2={res['n2_original']})")
            else:
                print(f"{res['group1']} vs {res['group2']}: Statystyka={res['statistic']:.2f}, p-wartość={res['p_value']:.4f}, Istotne={res['significant']}")
                print(f"  Próbki: Oryginalne (n1={res['n1_original']}, n2={res['n2_original']}), Po zrównoważeniu: n={res['n_sampled']}")
                print(f"  Średnie czasy (oryginalne): {res['group1']}={res['mean1_original']:.2f}s, {res['group2']}={res['mean2_original']:.2f}s")
                print(f"  Mediany czasów (oryginalne): {res['group1']}={res['median1_original']:.2f}s, {res['group2']}={res['median2_original']:.2f}s")
                print(f"  Średnie czasy (zsamplowane): {res['group1']}={res['mean1_sampled']:.2f}s, {res['group2']}={res['mean2_sampled']:.2f}s")
                print(f"  Mediany czasów (zsamplowane): {res['group1']}={res['median1_sampled']:.2f}s, {res['group2']}={res['median2_sampled']:.2f}s")
        
        print("\nSzczegółowe testy Wilcoxona z zrównoważonym samplingiem dla par relacji (Relationship_Name):")
        wilcoxon_results_rel = pairwise_wilcoxon_tests_balanced(track_df, 'Relationship_Name', 'Track_Time_Seconds', track_id, random_seed)
        for res in wilcoxon_results_rel:
            if np.isnan(res['p_value']):
                print(f"{res['group1']} vs {res['group2']}: Za mało danych (n1={res['n1_original']}, n2={res['n2_original']})")
            else:
                print(f"{res['group1']} vs {res['group2']}: Statystyka={res['statistic']:.2f}, p-wartość={res['p_value']:.4f}, Istotne={res['significant']}")
                print(f"  Próbki: Oryginalne (n1={res['n1_original']}, n2={res['n2_original']}), Po zrównoważeniu: n={res['n_sampled']}")
                print(f"  Średnie czasy (oryginalne): {res['group1']}={res['mean1_original']:.2f}s, {res['group2']}={res['mean2_original']:.2f}s")
                print(f"  Mediany czasów (oryginalne): {res['group1']}={res['median1_original']:.2f}s, {res['group2']}={res['median2_original']:.2f}s")
                print(f"  Średnie czasy (zsamplowane): {res['group1']}={res['mean1_sampled']:.2f}s, {res['group2']}={res['mean2_sampled']:.2f}s")
                print(f"  Mediany czasów (zsamplowane): {res['group1']}={res['median1_sampled']:.2f}s, {res['group2']}={res['median2_sampled']:.2f}s")
        
        # Zapisujemy wyniki
        results['age_groups'][track_id] = {
            'kruskal_x': kruskal_result_age_x,
            'kruskal_y': kruskal_result_age_y,
            'wilcoxon_x': wilcoxon_results_age_x,
            'wilcoxon_y': wilcoxon_results_age_y
        }
        
        results['relationships'][track_id] = {
            'kruskal': kruskal_result_rel,
            'wilcoxon': wilcoxon_results_rel
        }
    
    return results

# Funkcja do wizualizacji wyników
def visualize_results(df, results):
    # Analizujemy tylko ukończone podejścia
    completed_df = df[df['Track_Completed'] == True]
    
    # Unikalne identyfikatory torów
    track_ids = sorted(df['Track_ID'].unique())
    
    # Tworzymy wykresy pudełkowe dla każdego toru, pokazujące różnice między grupami wiekowymi
    plt.figure(figsize=(15, 10))
    for i, track_id in enumerate(track_ids):
        track_df = completed_df[completed_df['Track_ID'] == track_id]
        
        # Sprawdzamy, czy mamy wystarczającą ilość danych dla tego toru
        if len(track_df) < 5:
            continue
        
        plt.subplot(len(track_ids), 2, i*2+1)
        sns.boxplot(x='Age_Group_X', y='Track_Time_Seconds', data=track_df)
        plt.title(f'Tor {track_id} - Wpływ wieku uczestnika X')
        plt.ylabel('Czas [s]')
        plt.tight_layout()
        
        plt.subplot(len(track_ids), 2, i*2+2)
        sns.boxplot(x='Age_Group_Y', y='Track_Time_Seconds', data=track_df)
        plt.title(f'Tor {track_id} - Wpływ wieku uczestnika Y')
        plt.ylabel('Czas [s]')
        plt.tight_layout()
    
    plt.savefig('age_group_boxplots.png')
    plt.close()
    
    # Tworzymy wykresy pudełkowe dla każdego toru, pokazujące różnice między typami relacji
    plt.figure(figsize=(15, len(track_ids)*5))
    for i, track_id in enumerate(track_ids):
        track_df = completed_df[completed_df['Track_ID'] == track_id]
        
        # Sprawdzamy, czy mamy wystarczającą ilość danych dla tego toru
        if len(track_df) < 5:
            continue
        
        plt.subplot(len(track_ids), 1, i+1)
        sns.boxplot(x='Relationship_Name', y='Track_Time_Seconds', data=track_df)
        plt.title(f'Tor {track_id} - Wpływ relacji')
        plt.ylabel('Czas [s]')
        plt.xticks(rotation=45)
        plt.tight_layout()
    
    plt.savefig('relationship_boxplots.png')
    plt.close()

In [28]:
# Funkcja zapisująca analizy do pliku txt
def save_analysis_to_file(output_file='analiza_wynikow.txt', random_seed=42):
    # Przechwytujemy wszystkie wyniki analizy do bufora
    output_buffer = io.StringIO()
    with redirect_stdout(output_buffer):
        # Wczytanie danych
        df = load_data("games_with_relationship.csv")
        
        # Przygotowanie danych
        df = prepare_data(df)
        
        # Podstawowe statystyki
        print("Podstawowe statystyki:")
        print(f"Liczba wierszy: {len(df)}")
        print(f"Unikalne tory: {df['Track_ID'].unique()}")
        print(f"Unikalne grupy wiekowe X: {df['Age_Group_X'].unique()}")
        print(f"Unikalne grupy wiekowe Y: {df['Age_Group_Y'].unique()}")
        print(f"Unikalne relacje: {df['Relationship_Name'].unique()}")
        print(f"Procent ukończonych podejść: {(df['Track_Completed'] == True).mean() * 100:.2f}%")
        
        # Analiza wpływu wieku i relacji na wyniki z zrównoważonym samplingiem
        print("\nUWAGA: W testach Wilcoxona zastosowano zrównoważony sampling, aby zapewnić równą liczność grup.")
        results = analyze_performance(df, random_seed)
        
        # Wizualizacja wyników (nie wyświetlamy w pliku)
        print("\nWizualizacja wyników:")
        print("Wykresy zostały zapisane do plików 'age_group_boxplots.png' i 'relationship_boxplots.png'.")
        
        print("\nAnaliza zakończona.")
    
    # Zapisujemy wszystkie wyniki do pliku
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(output_buffer.getvalue())
    
    # Dodatkowo tworzymy wykresy
    visualize_results(df, results)
    
    print(f"Analiza została zapisana do pliku '{output_file}'.")
    print("Wykresy zostały zapisane do plików 'age_group_boxplots.png' i 'relationship_boxplots.png'.")

In [29]:
random_seed = 42
save_analysis_to_file(random_seed=random_seed)

Analiza została zapisana do pliku 'analiza_wynikow.txt'.
Wykresy zostały zapisane do plików 'age_group_boxplots.png' i 'relationship_boxplots.png'.
